In [32]:
import sys, Quandl,nltk, csv, urllib2, codecs, simplejson, decimal, string, json
import numpy as np
from pprint import pprint as pp
from nltk.corpus import movie_reviews
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
from pymongo import MongoClient
import pandas as pd
from operator import itemgetter
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
client = MongoClient()
db = client.dsbc
nw = db.neuro_words
nwfeq = db.nwfeq
nwfeq2 = db.nwfeq2

# nw.delete_many({"$where":"this.content.length < 100"})
# nw.delete_many({"$where":"typeof(this.content) != 'string'"})

# contents = nw.find({},{"content":1, "_id":0})
# content_vals = [i['content'] for i in contents]

len(content_vals)
with open('result3.json') as data_file:    
    data = json.load(data_file)

# for rev in reviews:
#     hmm.save(rev)


In [34]:
nwfeq2.remove({})
for key, val in data.items():
#     if index == 10:
#         break
    bregion = key.split(" | ")[1]
    scale = key.split(" | ")[0]
    my_hash = {"bregion": bregion, "scale": scale, "co-occ_count": val}
    nwfeq2.save(my_hash)

In [35]:
# cursor = nwfeq2.find({'scale': {"$ne" : "quest"}}).sort([("co-occ_count", -1)])
cursor = nwfeq2.find().sort([("co-occ_count", -1)])
# db.test.find({"number": {"$gt": 1}}).sort([("number", 1), ("date", -1)])
list(cursor)

[{u'_id': ObjectId('5567c2ebea6387acd09467bb'),
  u'bregion': u'frontal cortex',
  u'co-occ_count': 65,
  u'scale': u'Quest'},
 {u'_id': ObjectId('5567c2ebea6387acd094676d'),
  u'bregion': u'prefrontal cortex',
  u'co-occ_count': 40,
  u'scale': u'Quest'},
 {u'_id': ObjectId('5567c2ebea6387acd09467f8'),
  u'bregion': u'amygdala',
  u'co-occ_count': 33,
  u'scale': u'Quest'},
 {u'_id': ObjectId('5567c2ebea6387acd09467ce'),
  u'bregion': u'anterior cingulate',
  u'co-occ_count': 29,
  u'scale': u'Quest'},
 {u'_id': ObjectId('5567c2ebea6387acd09467de'),
  u'bregion': u'hippocampus',
  u'co-occ_count': 24,
  u'scale': u'Quest'},
 {u'_id': ObjectId('5567c2ebea6387acd09467a8'),
  u'bregion': u'cingulate cortex',
  u'co-occ_count': 23,
  u'scale': u'Quest'},
 {u'_id': ObjectId('5567c2ebea6387acd094678c'),
  u'bregion': u'thalamus',
  u'co-occ_count': 20,
  u'scale': u'Quest'},
 {u'_id': ObjectId('5567c2ebea6387acd094676c'),
  u'bregion': u'orbitofrontal cortex',
  u'co-occ_count': 17,
  u'sca

In [29]:
cursor_grouped = nwfeq2.aggregate(
   [
     { "$group" : { "_id" : "$co-occur_count", "nwfeq2": { "$push": "$scale" } } }
   ]
)
list(cursor_grouped)

[{u'_id': None,
  u'nwfeq2': [u'viewpoint',
   u'pace',
   u'viewpoint',
   u'quest',
   u'quest',
   u'feelings',
   u'quest',
   u'quest',
   u'quest',
   u'pace',
   u'pace',
   u'viewpoint',
   u'maslach burnout inventory',
   u'viewpoint',
   u'listening comprehension',
   u'quest',
   u'quest',
   u'pace',
   u'quest',
   u'pace',
   u'pace',
   u'quest',
   u'quest',
   u'stress audit',
   u'interpersonal adjective scales',
   u'motives',
   u'pace',
   u'viewpoint',
   u'quest',
   u'quest',
   u'viewpoint',
   u'pace',
   u'quest',
   u'viewpoint',
   u'clinical rating scale',
   u'interpersonal adjective scales',
   u'autobiographical memory interview',
   u'pace',
   u'maslach burnout inventory',
   u'viewpoint',
   u'quest',
   u'viewpoint',
   u'pace',
   u'composite international diagnostic interview',
   u'quest',
   u'rorschach',
   u'aggression questionnaire',
   u'feelings',
   u'pace',
   u'viewpoint',
   u'recognition memory test',
   u'quest',
   u'pace',
   u'stre

In [3]:
# vectorizer = TfidfVectorizer(stop_words="english", lowercase=False, encoding=u'utf-8')
# doc_vectors = vectorizer.fit_transform(content_vals)
# idf = vectorizer._tfidf.idf_
# idf_dic = dict(zip(vectorizer.get_feature_names(), idf))
# def format_brain(fname):
#     my_file = open(fname, "r")
#     read_file = my_file.read()
#     my_file.close()
#     return list(set(read_file.translate(string.maketrans("",""), string.punctuation).lower().split("\n")))

# def format_scale(fname):
#     my_file = open(fname, "r")
#     lines = list(set(my_file.read().split("\r")))
#     return [line.split(',')[0] for line in lines]

# bterms = format_brain("mesh_brain3.txt")
# sterms = format_scale("mmy_index.txt")

In [103]:
nw2 = db.neuro_words2
nw2.delete_many({"$where":"this.content.length < 100"})
nw2.delete_many({"$where":"typeof(this.content) != 'string'"})
thing = nw.find({},{"scales":1, "_id":0})
list(thing)

[{},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
